In [4]:
print("Starting chunk")

Starting chunk


<H2>Install and load required libraries</H2>

In [1]:
!conda install -c anaconda lxml --yes
!conda install -c anaconda bs4 --yes
!conda install -c conda-forge geocoder --yes
!conda install -c conda-forge geopy --yes
!pip install pgeocode
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
#!pip install pgeocode
import pgeocode
import lxml
import bs4
import geocoder
import geopy
import pandas
import numpy
import folium # map rendering library
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
from numpy import array
from pandas import Series
import pandas
from  geopy.geocoders import Nominatim

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - lxml


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.6.24  |                0         133 KB  anaconda
    certifi-2020.6.20          |           py36_0         160 KB  anaconda
    libxml2-2.9.10             |       he19cac6_1         1.3 MB  anaconda
    libxslt-1.1.34             |       hc22bd24_0         573 KB  anaconda
    lxml-4.5.2                 |   py36hefd8a0e_0         1.4 MB  anaconda
    openssl-1.1.1g             |       h7b6447c_0         3.8 MB  anaconda
    ------------------------------------------------------------
                                           Total:         7.3 MB

The following NEW packages will be INSTALLED:

  libxslt            anaconda/linux-64::libxslt-1.1.34-hc22bd24_0
  lxml           

<H2>Get Post code, Longitude and Latitude of each area in Winnipeg</H2>

In [2]:
nomi = pgeocode.Nominatim('CA')

# get html content for Canada post code
response = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_R")

soup = BeautifulSoup(response.text, 'lxml')
table = soup.find_all('table')[0]
row_marker = 0

ls = []

for row in table.find_all('tr'):

    columns = row.find_all('td')
    for column in columns:
        txt1 = column.get_text()
        pcode = txt1[0:3]
        area = txt1[3:].rstrip()
        if(not (area == "Not assigned") and ("Winnipeg" in area)):
            area = area.rstrip(')')[9:]
            tb1 = nomi.query_postal_code(pcode)
            latitude = tb1['latitude']
            longitude = tb1['longitude']
            ls.append([pcode,area,longitude,latitude])   

neighborhoods = pd.DataFrame(ls, columns=['Post Code', 'Area','Longitude', 'Latitude'])
neighborhoods.head()


,Post Code,Area,Longitude,Latitude
0,R3A,Centennial,-97.1489,49.9038
1,R3B,Chinatown / Civic Centre / Exchange District /...,-97.1401,49.8980
2,R2C,Transcona,-96.9563,49.9247
3,R3C,Broadway / The Forks / Portage and Main)Manito...,-97.2184,50.0110
4,R3E,Sargent Park / Daniel McIntyre / Inkster SE,-97.1779,49.9081


<H2>Get Asian restaurants for each area in Winnipeg from Foursquare location data server</H2>
<H2>Categorize areas by density of asian restaurant</H2>

In [3]:

category = "4bf58dd8d48988d142941735" #asian restaurant

url1='//api.foursquare.com/v2/venues/search?categoryId='+category

client_id='client_id=TZ4P15YOCGQGPMXOBZDKXSDCAAGXWDZ4JKRZLRLBYBOI3YGT'
client_secret='client_secret=OOEZFB045S32IW4QHDPCHGXW20D0WTDHKSGRP4JUTR0DHYGE'
v='v=20200101'
radius='radius=1000'
limit='limit=500'

count=0
df_merge=pandas.DataFrame()
area_list = []
for lat, lng, area in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Area']):
    count+=1
    ll='ll='+str(lat)+','+str(lng)
    url='&'.join([url1,ll,client_id,client_secret,v,radius,limit])

    result = requests.get("https:"+url).json()

    if "venues" in result['response'].keys():
        venues = result['response']['venues']
        dataframe = pandas.json_normalize(venues)
        area_list = area_list + [area] *(dataframe.shape[0])
        if(count==1):
            df_merge=dataframe
        else:
            df_merge = pd.concat([df_merge, dataframe], ignore_index=True)
    else:
        print(neighborhoods['Latitude']+neighborhoods['Longitude']+neighborhoods['Area'])
        
df_merge["Area"] = area_list

asian_df = df_merge[['name','location.lat','location.lng','location.postalCode','Area']]
asian_count_by_area = asian_df['Area'].value_counts().to_dict()
mean_restaurant = int(Series([*asian_count_by_area.values()]).mean())
print("Average count of asian restaurant of each area")
print(mean_restaurant)
print("Count of asian restaurant of each area")
print(asian_count_by_area)

area_category=pandas.DataFrame()
count = 0
for area in neighborhoods['Area']:
    
    count += 1
    if area not in asian_count_by_area:
        col_value = "zero"
    else:
        if asian_count_by_area[area] > mean_restaurant:
            col_value = "high density"
        else:
            col_value = "low density"
   
    df2 = pd.DataFrame([[area, col_value]], columns=list(['area',"density"]))
    if count == 1:
        area_category = df2
    else:
        area_category = pd.concat([area_category, df2], ignore_index=True)
area_category = area_category.sort_values(by=['density'],ascending=False)
print("=======")
print("Density of asian restaurant by area")
print(area_category)

Average count of asian restaurant of each area
11
Count of asian restaurant of each area
{'Centennial': 50, 'Chinatown / Civic Centre / Exchange District / University of Winnipeg': 49, 'Fort Garry NE / University of Manitoba': 23, 'Sargent Park / Daniel McIntyre / Inkster SE': 19, 'St. Boniface NW': 14, 'River Heights East': 11, 'Minto / St. Mathews / Wolseley': 11, 'Crescentwood Central': 11, 'St. Vital North': 11, 'River Heights West': 6, 'Fort Garry NW / Tuxedo': 6, 'Point Douglas East': 6, 'East Kildonan': 4, 'Elmwood': 4, 'St. James-Assiniboia SW': 4, 'Point Douglas West / Inkster East': 3, 'St. James-Assiniboia NE / YWG': 3, 'St. Boniface NE': 1, 'North Kildonan North': 1, 'Assiniboine South / Betsworth': 1}
Density of asian restaurant by area
                                                 area       density
32                                    Fort Garry West          zero
28                   Grassie / Eastern North Kildonan          zero
19                                  

<H2>Get Winnipeg longitude and latitude for creating location map</H2>

In [4]:
#Search Winnipeg
geolocator = Nominatim(user_agent="dnipro")
location = geolocator.geocode("Winnipeg")
print("get location latitude and longitude of Winnipeg")
print((location.latitude, location.longitude))
latitude = location.latitude
longitude = location.longitude


get location latitude and longitude of Winnipeg
(49.8955367, -97.1384584)


<H2>Create Winnipeg Map and mark each area  </H2>
<ul>
<li><font color=purple>Area with no asian restaurants</font></li>
<li><font color=red>Area with below average number of restaurants</font></li>
<li><font color=blue>Area with above average number of restaurants</font></li>
</ul>

In [5]:
# create map of Winnipeg using latitude and longitude values
map_winnipeg = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, area in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Area']):
    label = '{}'.format(area)
    label = folium.Popup(label, parse_html=True)
    if area not in asian_count_by_area:
        col_value = "purple"
    else:
        if asian_count_by_area[area] > mean_restaurant:
            col_value = "blue"
        else:
            col_value = "red"      
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color=col_value,
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_winnipeg)  
    
map_winnipeg

<H2>Recommendation of opening an asian restaurant in an area with zero or low density</H2>

In [6]:
area_recommendation = area_category.loc[(area_category['density'] == "zero") | (area_category['density'] == "low density")]
print(area_recommendation)

                                                 area      density
32                                    Fort Garry West         zero
28                   Grassie / Eastern North Kildonan         zero
19                                       Old Kildonan         zero
21                                       Inkster West         zero
23                                       Wilkes South         zero
25                                      West Kildonan         zero
10                            St. James-Assiniboia SE         zero
26                                   Fort Garry South         zero
30                  St. Boniface South / St. Vital SE         zero
31                            St. James-Assiniboia NW         zero
3   Broadway / The Forks / Portage and Main)Manito...         zero
2                                           Transcona         zero
17                                       St. Vital SW         zero
27                                 Point Douglas East  low den

<H2>Create Winnipeg Map and mark recommendation areas  </H2>
<ul>
<li><font color=purple>Area with no asian restaurants</font></li>
<li><font color=red>Area with below average number of restaurants</font></li>
</ul>

In [7]:
# create map of Winnipeg using latitude and longitude values
map_winnipeg = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, area in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Area']):
    label = '{}'.format(area)
    label = folium.Popup(label, parse_html=True)
    selection = True
    if area not in asian_count_by_area:
        col_value = "purple"
    else:
        if asian_count_by_area[area] > mean_restaurant:
            col_value = "blue"
            selection = False
        else:
            col_value = "red"  
    if selection:
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=col_value,
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_winnipeg)  
    
map_winnipeg